# Deferred Feedback Evaluation

Running feedback in "deferred" mode allows them to be computed by a separate process or even computer as long as it has access to the same database as the `TruSession` object. In this notebook we demonstrate how to set this up.

In [ ]:
from pathlib import Path
import sys

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
import numpy as np
from trulens.apps.custom import TruCustomApp
from trulens.core import Feedback
from trulens.core import Select
from trulens.core import TruSession
from trulens.core.schema import FeedbackMode
from trulens.core.utils.threading import TP
from trulens.dashboard import run_dashboard
from trulens.providers.huggingface.provider import Dummy

from examples.dev.dummy_app.app import DummyApp

tp = TP()

d = Dummy(
    loading_prob=0.0,
    freeze_prob=0.0,
    error_prob=0.0,
    overloaded_prob=0.0,
    rpm=6000,
)

session = TruSession()

session.reset_database()

run_dashboard(session, force=True)

In [ ]:
# Set up some feedback functions based on the dummy provider as well as the
# example dummy app.

f_dummy_min = (
    Feedback(
        d.positive_sentiment,
        name="min aggregate",
    )
    .on(text=Select.Record.main_output[::20])
    .aggregate(np.min)
)

f_dummy_max = (
    Feedback(d.positive_sentiment, name="max aggregate")
    .on(text=Select.Record.main_output[::20])
    .aggregate(np.max)
)


# Create custom app:
ca = DummyApp()

In [ ]:
# Create trulens wrapper with the previously defined feedback functions,
# specifying `feedback_mode`.

ta = TruCustomApp(
    ca,
    app_name="customapp",
    feedbacks=[f_dummy_min, f_dummy_max],
    feedback_mode=FeedbackMode.DEFERRED,  # deferred feedback mode
)

In [ ]:
# Run the app. This will not produce any feedbacks but will add them to the
# database for the deferred evaluator to run them later.

with ta as recorder:
    ca.respond_to_query("hello")

In [ ]:
# Start the deferred feedback evaluator. This is a non-blocking call. If you are
# running this in a separate process, make sure you don't exit.
session.start_evaluator(restart=True)